### Basic Claude3 API Operations

- Tutorials
    - Intro to Claude Prompting: https://docs.anthropic.com/claude/docs/intro-to-prompting
    - Metaprompt helper: https://docs.anthropic.com/claude/docs/helper-metaprompt-experimental
    
- 3 Claude Variants(context window): version (ranked from smallest to largest)
    - Haiku(200k): claude-3-haiku-20240307
    - Sonnet(200k): claude-3-sonnet-20240229
    - Opus(200k): claude-3-opus-20240229
    - Claude2.1(200k): claude-2.1
    - Claudee2(100k): claude-2.0
    
- limitations
    - Cannot open Links
    - 

In [1]:
import os
import anthropic
from IPython.display import Markdown, display
from docx import Document

import os
from pathlib import Path

_root = Path.cwd()
_utils_dir = _root.parent.parent.as_posix()  #'/home/zjc1002/Mounts/code/MyModules/utils
os.sys.path.append(_utils_dir)
from utils.misc import LoadCFG
config_path = (_root / "config.yaml").as_posix()
cfg = LoadCFG(config_path, base_dir=_root.as_posix(), return_namespace=False).load()


#Anthropic API config params
api_key = cfg['api_key'] #my api key
model = cfg['model'] #name of anthropic model to use 
max_tokens = cfg['max_tokens'] #max tokens in anthropic model response 
temperature = cfg['temperature'] #how diverse do you want to the responses to be?

File DOES exist:
	 /home/zjc1002/Mounts/code/MyModules/genai/claude3_ops/config.yaml


In [2]:
def load_docx_text(file_path):
    doc = Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

### Create client object using API key. 
*This client can be used for text generation , access vision capabliity, and streaming* 

In [3]:
client = anthropic.Anthropic(api_key=api_key)

#### Claude Client Request 

- Configurable Parameters 
    - Temperature: used to control the randomness of the generated response from the model.

### Basic Prompt Examples 


- Useful techniques 
    - **Stop Sequences** - A stop sequence is a string that stops the model from generating tokens. Specifying stop sequences is another way to control the length and structure of the model's response. For example, you can tell the model to generate lists that have no more than 10 items by adding "11" as a stop sequence.

In [25]:
#create a function to identify all active network connections 
message = client.messages.create(
    model=model
    ,max_tokens=max_tokens
    ,temperature=temperature
    ,system="you are a AI coding expert with a keen eye for simplicity. Your task is to create a python function to identify all active network connections on a local computer. The function should return a list of dictionaries, each containing the following keys: 'local_address', 'local_port', 'remote_address', 'remote_port', 'state', 'pid', 'process_name'."
    ,messages=[
        {"role": "user", "content": "write a python function to identify all active network connections on a local computer?"}
    ]
)
print(message.content)

[ContentBlock(text='Here\'s a Python function that identifies all active network connections on a local computer and returns a list of dictionaries containing the connection details:\n\n```python\nimport psutil\nimport socket\n\ndef get_active_connections():\n    connections = []\n    \n    for conn in psutil.net_connections():\n        if conn.status == psutil.CONN_ESTABLISHED or conn.status == psutil.CONN_SYN_SENT:\n            try:\n                process = psutil.Process(conn.pid)\n                process_name = process.name()\n            except psutil.NoSuchProcess:\n                process_name = "Unknown"\n            \n            local_address = conn.laddr.ip\n            local_port = conn.laddr.port\n            remote_address = conn.raddr.ip if conn.raddr else None\n            remote_port = conn.raddr.port if conn.raddr else None\n            \n            connection = {\n                \'local_address\': local_address,\n                \'local_port\': local_port,\n     

#### CoverLetter Revisor
*define a prompt to effectivly refine a coverletter to better represent the domain and skillsets required for the role. The agent works to both simply the language and improve the domain relevance of the input coverletter*

In [4]:
# Define our input prompt template for the task. (zjc 3/15)
def build_input_prompt(coverletter):
    
    user_content = f"""You will be provided a coverletter for a data science managerial position at a financial instutution. 
    Your job consists of 2 components: 1) Improve the quality of the coverletter based on your knowlodge of how data science and generative AI is applied within financial instututions. 2) Simplify the coverletter by removing duplicate or unnecessary information and correcting grammatical errors.  
          
    Specifically your goal is to improve the semantic flow of the coverletter to better represent the requirments of a data science manager working within the financial instrustry. You should also ensure the coverletter is free of any grammatical errors. You can use the following as a guide to help you evaluate the coverletter: 

    1) Does the coverletter effectively communicate the candidate's experience and skills in data science?
    2) Does the coverletter effectively communicate the candidate's understanding of the financial industry?
    3) Does the coverletter effectively communicate the candidate's understanding of the role of a data science manager?
    4) Does the coverletter effectively describe the candidate as a leader that can sucessfuly manage a team of data scientists?

    Here is the coverletter.
    <coverletter>{coverletter}</coverletter>
    
    Please produce a revised version of the coverletter with your expert revisions. Do not include any comments or feedback in your response. Your response cannot exceed 4000 tokens."""

    
    messages = [{'role': 'user', 'content': user_content}]
    return messages


# Define our get_completion function (including the stop sequence discussed above(Your response cannot exceed 4000 tokens.)).
def get_completion(messages, model=None, max_tokens = 5):
    response = client.messages.create(
        model=model,
        max_tokens=max_tokens,
        messages=messages
    )
    return response.content[0].text

In [ ]:
#get coverletter text
file_path = "/home/zjc1002/Mounts/data/coverletter/Zachary Carideo CoverLetter 2024.docx"
coverletter_text = load_docx_text(file_path)

# Get completions for each input.
output = get_completion(build_input_prompt(), model=model)
#print it out 
print(output.content[0].text)

### Improve the grammmer and flow of any sentence/paragraph or chunk of text 

In [23]:
_text = "Throughout my experience as a manager and data scientist, I have learned the key to generating value lies in a manager's ability to foster a high-performing team environment and  strategize with leaders across business units to identify new opportunities to drive value and mitigate operational risks through the implementation of AI/ML initiatives. "

# Define our input prompt template for the task. (zjc 3/15)
def build_input_prompt(text):
    
    user_content = f"""You will be provided a peice of text. Your job consists of 2 components: 1) Improve the grammer of the text , 2) re-phrase the text in a simplified manner that consolidates the original text.   
    <text>{text}</text>
    Please produce a revised version of the text with your expert revisions. Do not include any comments or feedback in your response."""

   
    messages = [{'role': 'user', 'content': user_content}]
    return messages


# Define our get_completion function (including the stop sequence discussed above(Your response cannot exceed 4000 tokens.)).
def get_completion(messages, model=None, max_tokens = 5,temperature=temperature):
    response = client.messages.create(
      model=model,
      max_tokens=max_tokens,
      temperature=temperature,
      system="Respond in short and clear sentences.",      
      messages=messages
    )
    return response#.content[0].text

In [27]:
# Get completions for each input and copy to clipboard.
import pyperclip
output = get_completion(build_input_prompt(_text), model=model, max_tokens=int(len(_text)/5), temperature=.6)
#print it out 
print(output.content[0].text)
pyperclip.copy(output.content[0].text)

Throughout my career as a manager and data scientist, I have discovered that creating value depends on two critical factors. First, a manager must cultivate a high-performing team environment. Second, they must collaborate with leaders across various business units to pinpoint new opportunities. By doing so, they can drive value and reduce operational risks through the implementation of AI


#### Summarizing Web Page Content

In [45]:
import requests, json
def download(url):

    response = requests.get(url)

    if response.status_code == 200:
        page_content = response.text
    else:
        print(f"Failed to fetch the web page. Status code: {response.status_code}")
        exit(1)
    return page_content        
    

url = "https://en.wikipedia.org/wiki/96th_Academy_Awards"
page_content = download(url)


#prepare input for claude
prompt = f"<content>{page_content}</content>Please produce a concise summary of the web page content. The summary cannot be longer than 500 tokens"
messages = [{"role": "user", "content": prompt}]

#generate the summary 
response = client.messages.create(
    model=model ,
    max_tokens=500,
    messages=messages
)

summary = response.content[0].text
print(summary)

The 96th Academy Awards ceremony, presented by the Academy of Motion Picture Arts and Sciences (AMPAS), took place on March 10, 2024 at the Dolby Theatre in Los Angeles. Comedian Jimmy Kimmel hosted the show for the fourth time.

The nominations were announced on January 23, 2024, with Oppenheimer leading with 13 nominations, followed by Poor Things and Killers of the Flower Moon with 11 and 10, respectively. Oppenheimer won a leading seven awards, including Best Picture and Best Director.

Notable nominees included Steven Spielberg, Martin Scorsese, Thelma Schoonmaker, composer John Williams, and Willie D. Burton. Several actors received their first Oscar nominations. The ceremony also featured the revival of a popular presenting format where five Oscar-winning actors took the stage together to introduce the current nominees in their respective categories.

The telecast drew 19.5 million viewers in the United States, becoming the most watched awards show since 2020. The ceremony recei

### Returning JSON From Claude

In [47]:
from anthropic import Anthropic
import json
import re, json 
from pprint import pprint

def extract_json(response):
    json_start = response.index("{")
    json_end = response.rfind("}")
    return json.loads(response[json_start:json_end + 1])

what_i_want = 'top 10 most expensive zipcodes in Charlotte, NC including there average home price'

#incorporate json request into prompt to tell claude you want a json formatted response
message = client.messages.create(
    model=model,
    max_tokens= 1000,
    messages=[
        {
            "role": "user", 
            "content": f"Give me a JSON dict with {what_i_want}"
        },
    ]
).content[0].text



{'28207': {'average_home_price': 1450000}, '28209': {'average_home_price': 1150000}, '28211': {'average_home_price': 950000}, '28204': {'average_home_price': 850000}, '28203': {'average_home_price': 800000}, '28202': {'average_home_price': 750000}, '28226': {'average_home_price': 700000}, '28270': {'average_home_price': 650000}, '28277': {'average_home_price': 600000}, '28105': {'average_home_price': 550000}}


### Meta Prompt Helper 
*The metaprompt is particularly useful as a "getting started" tool or as a method to generate multiple prompt versions for a given task, making it easier to test a variety of initial prompt variations for your use case.*
- https://github.com/anthropics/anthropic-cookbook/blob/main/misc/building_moderation_filter.ipynb
- https://console.anthropic.com/settings/plans
